# Notes
- Different species
- Gap phase replacement
- Spatial field

# To-do
- Implement plant reproduction (either as a method in the Plant class or in the Simulation class)
- Write your own quadtree class and add a method for removing objects from it to speed up collision detection (removing dead plants before checking collisions)
- Inform initial spatial and size distributions
- Inform reproductive mechanism (range, probability, etc.)
- Inform mortality mechanism
- Fix new plant spawns showing up for one frame and then getting killed
- ~~Add find collisions to Simulation class~~
- ~~Move plants list and simulation_states list to Simulation class~~
- ~~Fix Plant .kill() method~~
- ~~replace .update_color() method to set_color() in Plant class~~
- ~~add simulation.step() method to Simulation class~~

In [3]:
import numpy as np
a = np.array([0.1, 0.2])

array([[0.1, 0.2]])